In [36]:
from libs.llm_chat import llama_3_1_8b, create_chain, get_session_history


In [34]:
# chain_hermes = create_chain(model=hermes)
chain = create_chain(model=llama_3_1_8b)

In [32]:
user_id = 'саша003'
question = 'Привет, какой КБЖУ у онигири?'

In [37]:
response_content = chain.invoke({"input": question}, config={"configurable": {"session_id": user_id}})
response_content

TypeError: PostgresChatMessageHistory.__init__() got some positional-only arguments passed as keyword arguments: 'table_name, session_id'

In [13]:
session_history = get_session_history(user_id)
session_history

In [18]:
from langchain_core.messages import HumanMessage, AIMessage

session_history = get_session_history(user_id)

# Преобразование сообщений в список словарей
messages = []
for message in session_history.messages:
    if isinstance(message, HumanMessage):
        messages.append({"Human": message.content})
    elif isinstance(message, AIMessage):
        messages.append({"AI": message.content})
messages

[{'Human': 'Привет! Когда приедет курьер?'},
 {'AI': 'Привет! :) Время доставки заказа зависит от выбранного интервала и адреса доставки. Точное время прибытия курьера будет указано в приложении после оформления заказа. Если у вас возникнут вопросы или необходима помощь, обращайтесь в наш Живой Чат, и мы с радостью поможем!'}]

In [1]:
from langchain.chains.retrieval import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.runnables import RunnableWithMessageHistory
from pathlib import Path
import os
from dotenv import load_dotenv
from langchain_community.chat_models import GigaChat
from libs.llm_chat import (TEMPERATURE, MAX_TOKENS, VEC_STORE_LOAD_PATH, define_promt, get_history_aware_retriever,
                           get_session_history)


def create_chain_gigachat(vec_store_path: str | Path = VEC_STORE_LOAD_PATH) -> RunnableWithMessageHistory:
    """
        Создает и возвращает цепочку обработки запросов с учетом истории чата.

    Returns:
        RunnableWithMessageHistory: Цепочка обработки запросов с учетом истории чата.
    """
    load_dotenv()
    GIGACHAT_KEY = os.environ.get('GIGACHAT_KEY')
    
    chat = GigaChat(credentials=GIGACHAT_KEY, 
                    verify_ssl_certs=False,
                    scope="GIGACHAT_API_PERS",
                    model='GigaChat', # попробовать GigaChat-Pro
                    temperature=TEMPERATURE,
                    max_tokens=MAX_TOKENS)
    
    prompt = define_promt()

    doc_chain = create_stuff_documents_chain(chat, prompt)
    history_aware_retriever = get_history_aware_retriever(chat, vec_store_path)

    chain = create_retrieval_chain(history_aware_retriever, doc_chain)

    # Create a chain
    conversational_rag_chain = RunnableWithMessageHistory(
        chain,
        get_session_history,
        input_messages_key="input",
        history_messages_key="chat_history",
        output_messages_key="answer",
    )

    return conversational_rag_chain

In [2]:
chain_gigachat = create_chain_gigachat()
chain_gigachat

C:\D\livemart\chat_bot_lm\.venv\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


RunnableWithMessageHistory(bound=RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  chat_history: RunnableBinding(bound=RunnableLambda(_enter_history), config={'run_name': 'load_history'})
}), config={'run_name': 'insert_history'})
| RunnableBranch(branches=[(RunnableBinding(bound=RunnableLambda(_is_not_async), config={'run_name': 'RunnableWithMessageHistoryInAsyncMode'}), RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
    context: RunnableBinding(bound=RunnableBranch(branches=[(RunnableLambda(lambda x: not x.get('chat_history', False)), RunnableLambda(lambda x: x['input'])
             | VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001E4D15BC310>))], default=ChatPromptTemplate(input_variables=['chat_history', 'input'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, lang

In [5]:
import uuid

# user_id = 'саша004'
user_id = str(uuid.uuid4())
question = 'Привет \nКак получить бесплатный кофе?'
user_id

'ded07b43-933d-45bd-a9f7-cc197f97cddc'

In [6]:
response_content = chain_gigachat.invoke({"input": question}, config={"configurable": {"session_id": user_id}})
response_content

Error in RootListenersTracer.on_chain_end callback: KeyError('answer')
Error in callback coroutine: KeyError('answer')


{'input': 'Привет \nКак получить бесплатный кофе?',
 'chat_history': [],
 'context': [Document(page_content='Гость: Какой кофе мне могут сделать бесплатно по карточке?  \nБот: Здравствуйте! На главном экране в приложен ии Жизньмарт  появился раздел \nКофе в подарок, в котором есть копилка. Нажав на акцию с кофе, вы увидите, какие \nнапитки можно получить бесплатно 💚'),
  Document(page_content='Гость: Как сменить данные в профиле?  \nБот: Здравствуйте! Для этого перейдите в профиль в приложении или на сайте в \nЛичный Кабинет и внесите нужные изменения. После чего сохраните.'),
  Document(page_content='Гость: Как связаться с курьером?  \nБот: Здравствуйте! В приложении в разделе «Наши магазины» и на сайте в разделе \n«Адреса магазинов кафе» представлены все номера магазинов – вы можете \nпозвонить напрямую супергероям, а они сориентируют вас по коммуникации с \nкурьером :)'),
  Document(page_content='Гость: Как сменить номер в приложении?  \n"Бот: Здравствуйте! Один номер предназначен д